In [18]:
import numpy as np, pandas as pd, polars as pl
from sklearn.model_selection import KFold, GridSearchCV
from sklearn import preprocessing as skp
from sklearn.decomposition import PCA 
import seaborn as sns

import xgboost as xgb
from sklearn.metrics import root_mean_squared_error as rmse
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('train.csv', index_col = 'id')
test = pd.read_csv('test.csv', index_col = 'id')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 750000 entries, 0 to 749999
Data columns (total 11 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Podcast_Name                 750000 non-null  object 
 1   Episode_Title                750000 non-null  object 
 2   Episode_Length_minutes       662907 non-null  float64
 3   Genre                        750000 non-null  object 
 4   Host_Popularity_percentage   750000 non-null  float64
 5   Publication_Day              750000 non-null  object 
 6   Publication_Time             750000 non-null  object 
 7   Guest_Popularity_percentage  603970 non-null  float64
 8   Number_of_Ads                749999 non-null  float64
 9   Episode_Sentiment            750000 non-null  object 
 10  Listening_Time_minutes       750000 non-null  float64
dtypes: float64(5), object(6)
memory usage: 68.7+ MB


In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 250000 entries, 750000 to 999999
Data columns (total 10 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Podcast_Name                 250000 non-null  object 
 1   Episode_Title                250000 non-null  object 
 2   Episode_Length_minutes       221264 non-null  float64
 3   Genre                        250000 non-null  object 
 4   Host_Popularity_percentage   250000 non-null  float64
 5   Publication_Day              250000 non-null  object 
 6   Publication_Time             250000 non-null  object 
 7   Guest_Popularity_percentage  201168 non-null  float64
 8   Number_of_Ads                250000 non-null  float64
 9   Episode_Sentiment            250000 non-null  object 
dtypes: float64(4), object(6)
memory usage: 21.0+ MB


In [5]:
categorical_map = {
        'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6, # Day

        'Morning': 0, 'Afternoon': 1, 'Evening': 2, 'Night': 3,                                            # Time

        'Negative': 0, 'Neutral': 1, 'Positive': 2,                                                        # Sentiment

        'True Crime': 0, 'Comedy': 1, 'Education': 2, 'Technology': 3, 'Health': 4, 'News': 5, 'Music': 6, # Genre
        'Sports': 7, 'Business': 8, 'Lifestyle': 9
        }

categories = ['Genre', 'Podcast_Name', 'Episode_Title', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']

In [6]:
podcast = {}
for i, n in enumerate(train['Podcast_Name'].unique()):
    podcast[n] = i

categorical_map.update(podcast)

In [7]:
train['Episode_Title'] = train['Episode_Title'].str[8:]
test['Episode_Title'] = test['Episode_Title'].str[8:]

In [8]:
train = train.replace(categorical_map).fillna(0)
test = test.replace(categorical_map).fillna(0)

In [9]:
x = train.drop('Listening_Time_minutes', axis=1)
y = train['Listening_Time_minutes']

In [10]:
poly = skp.PolynomialFeatures(interaction_only=True, include_bias=False)
x_poly = poly.fit_transform(x)
test_poly = poly.fit_transform(test)

In [11]:
cols = poly.get_feature_names_out(x.columns)
x = pd.DataFrame(x_poly, columns=cols)

test_cols = poly.get_feature_names_out(test.columns)
test = pd.DataFrame(test_poly, columns=test_cols)

In [12]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 55 columns):
 #   Column                                                  Non-Null Count   Dtype  
---  ------                                                  --------------   -----  
 0   Podcast_Name                                            750000 non-null  float64
 1   Episode_Title                                           750000 non-null  float64
 2   Episode_Length_minutes                                  750000 non-null  float64
 3   Genre                                                   750000 non-null  float64
 4   Host_Popularity_percentage                              750000 non-null  float64
 5   Publication_Day                                         750000 non-null  float64
 6   Publication_Time                                        750000 non-null  float64
 7   Guest_Popularity_percentage                             750000 non-null  float64
 8   Number_of_Ads           

In [13]:
important_features_bygain = ['Episode_Length_minutes','Episode_Title Episode_Length_minutes', 'Episode_Title', 'Podcast_Name','Number_of_Ads',
'Episode_Length_minutes Host_Popularity_percentage', 'Podcast_Name Episode_Length_minutes', 'Host_Popularity_percentage']
idx = []
for i, col in enumerate(x.columns): 
    if col in important_features_bygain: idx.append(i)

In [14]:
x_n = x.iloc[idx]
test_n = test.iloc[idx]

In [15]:
for c in categories: 
    x_n[c] = x_n[c].astype('category')
    test_n[c] = test_n[c].astype('category')

C:\Users\evan\AppData\Local\Temp\ipykernel_20696\4176405769.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_n[c] = x_n[c].astype('category')
C:\Users\evan\AppData\Local\Temp\ipykernel_20696\4176405769.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_n[c] = test_n[c].astype('category')


In [17]:
pred = pd.DataFrame()
pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame


In [ ]:
#K = 5
#kf = KFold(K, shuffle=True, random_state=55)

# Remove KFold for now.

----

In [20]:
model = xgb.XGBRegressor(objective='reg:squarederror', eval_metric = 'rmse', enable_categorical = True, device = 'cuda', tree_method="hist")

In [23]:
parameters = {
    'learning_rate' : [0.1, 0.2],
    'min_child_weight' : [1,2,3,4,5,6,7]
}

In [ ]:
gs = GridSearchCV(model, parameters, scoring='neg_root_mean_squared_error', cv=5, verbose=2)
gs.fit(x, y)

In [31]:
for train_idx, val_idx in kf.split(x,y):
    x_t, y_t = x.iloc[train_idx], y.iloc[train_idx]
    x_v, y_v = x.iloc[val_idx], y.iloc[val_idx]
    model = xgb.XGBRegressor(objective='reg:squarederror', eval_metric = 'rmse', enable_categorical = True, device = 'cuda', tree_method="hist")#, min_child_weight=4)
    model.fit(x_t, y_t, eval_set=[(x_v,y_v)])
    pred = pd.concat([pd.DataFrame(model.predict(test)), pred], axis=1)

[0]	validation_0-rmse:21.24222
[1]	validation_0-rmse:17.63194
[2]	validation_0-rmse:15.54909
[3]	validation_0-rmse:14.41017
[4]	validation_0-rmse:13.81063
[5]	validation_0-rmse:13.50164
[6]	validation_0-rmse:13.34275
[7]	validation_0-rmse:13.25999
[8]	validation_0-rmse:13.21413
[9]	validation_0-rmse:13.18888
[10]	validation_0-rmse:13.17235
[11]	validation_0-rmse:13.16241
[12]	validation_0-rmse:13.15340
[13]	validation_0-rmse:13.14676
[14]	validation_0-rmse:13.14190
[15]	validation_0-rmse:13.13866
[16]	validation_0-rmse:13.13617
[17]	validation_0-rmse:13.13049
[18]	validation_0-rmse:13.12786
[19]	validation_0-rmse:13.12560
[20]	validation_0-rmse:13.12150
[21]	validation_0-rmse:13.11839
[22]	validation_0-rmse:13.11705
[23]	validation_0-rmse:13.11761
[24]	validation_0-rmse:13.11704
[25]	validation_0-rmse:13.11560
[26]	validation_0-rmse:13.11434
[27]	validation_0-rmse:13.10911
[28]	validation_0-rmse:13.10805
[29]	validation_0-rmse:13.10664
[30]	validation_0-rmse:13.10567
[31]	validation_0-

c:\Users\evan\anaconda3\lib\site-packages\xgboost\core.py:729: UserWarning: [17:40:34] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


[0]	validation_0-rmse:21.16720
[1]	validation_0-rmse:17.57018
[2]	validation_0-rmse:15.49624
[3]	validation_0-rmse:14.36435
[4]	validation_0-rmse:13.77050
[5]	validation_0-rmse:13.46701
[6]	validation_0-rmse:13.31094
[7]	validation_0-rmse:13.23013
[8]	validation_0-rmse:13.18720
[9]	validation_0-rmse:13.16134
[10]	validation_0-rmse:13.14582
[11]	validation_0-rmse:13.13775
[12]	validation_0-rmse:13.12993
[13]	validation_0-rmse:13.12564
[14]	validation_0-rmse:13.12308
[15]	validation_0-rmse:13.11695
[16]	validation_0-rmse:13.11184
[17]	validation_0-rmse:13.11050
[18]	validation_0-rmse:13.10641
[19]	validation_0-rmse:13.10463
[20]	validation_0-rmse:13.10004
[21]	validation_0-rmse:13.09814
[22]	validation_0-rmse:13.09736
[23]	validation_0-rmse:13.09608
[24]	validation_0-rmse:13.09300
[25]	validation_0-rmse:13.09091
[26]	validation_0-rmse:13.08745
[27]	validation_0-rmse:13.08648
[28]	validation_0-rmse:13.08630
[29]	validation_0-rmse:13.08531
[30]	validation_0-rmse:13.08343
[31]	validation_0-

In [32]:
pred.reset_index(drop=True, inplace=True)
pred.index += 750000
pred_sum = pred.sum(axis=1) / K

In [33]:
pred.head()

,0,0,0,0,0
750000,54.861084,55.457115,54.480507,54.700565,55.267632
750001,18.402489,17.822088,17.969061,18.139446,18.363321
750002,50.714630,49.588593,50.344738,49.929829,50.329323
750003,76.516151,76.128929,78.639053,77.351852,81.148071
750004,49.242504,48.899967,48.827999,48.878304,49.277840


In [34]:
pred_sum.rename('Listening_Time_minutes', inplace=True)
pred_sum.head()

750000    54.953381
750001    18.139280
750002    50.181419
750003    77.956810
750004    49.025322
Name: Listening_Time_minutes, dtype: float32

In [35]:
pred_sum.to_csv('submission_2.csv', index_label='id')